In [146]:
import os
import re
import shutil #directory controls
import json
import yaml
import shutil
import fnmatch
import errno
import itertools

# Snowflake DBT to Dataform Converter

### Edit variables below cell with the references of your project

[Roadmap convertion spreadsheet](https://docs.google.com/spreadsheets/d/1q96HottHJaEC9vZ0NPrwLsoxVpnRE7fLGz5zI-y0mM0/edit#gid=0).

In [147]:
#dbt source project path
dbt_source_project_path = "../../dbt/dbt_project_name/"

#dataform path to be created
dataform_root_path = "../_dataform_output"

#target schema on snowflake
target_schema = 'TARGET_SCHEMA_NAME'

#timestamp updated_at field to be used on snapshot convertion. 
#This field must represent the column name for the last updated date of the record
dlh_timestamp_field = '"MD_ELT_UPDATED_DTS"'

In [148]:
#creating variables based on defined paths

dbt_models_file_path = dbt_source_project_path+"/models/" 
dbt_snapshots_file_path = dbt_source_project_path+"/snapshots/" 

dataform_output_sources_path = dataform_root_path+"/definitions/sources"
dataform_output_models_path = dataform_root_path+"/definitions"

#JSON file of dataform connection to snowflake. https://docs.dataform.co/dataform-cli#create-a-credentials-file
dataform_credentials_file_path = "../.df-credentials.json"

In [149]:
#find files on directories and subdirectories
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

In [150]:
#YAML to dictionary
def read_yaml_file(filename):
    with open(filename, 'r') as stream:
        try:
            yaml_dict = yaml.load(stream, Loader=yaml.FullLoader)
            return yaml_dict
        except yaml.YAMLError as exc:
            print(exc)

In [151]:
#Function used to convert source (YML) to (SQLX) file
def create_sqlx_json_source_file(models_file_path,dataform_output_sources_path):

#creating directory
    isExist = os.path.exists(dataform_output_sources_path)
    if not isExist:
        os.makedirs(dataform_output_sources_path)

#iterate through each yaml file that contains sources
    for file in models_file_path:
        #reading all YAML files on source directory
        dic = read_yaml_file(file)
        #getting only sources entity from yaml file
        if 'sources' in dic.keys():

            sources = dic["sources"]
            
            for source in sources:
                #declaring variables for each element of dictionary that will be used. tables = list, schema and database = str
                tables = source["tables"]            
                database = source["database"]
                schema = source["schema"]

                #iterating through all tables for each schema
                for tables in tables:
                        #getting tables from list
                        tables= tables['name']                                
                        #creating json .sqlx structure
                        jmodel = "{\n"
                        jmodel += f'"type": "declaration",\n'                
                        jmodel += f'"database": "{database}",\n'
                        jmodel += f'"schema": "{schema}",\n'
                        jmodel += f'"name": "{tables}"'
                        jmodel += "\n}" #close out JSON file EOF

                        #parsing json and replacing double quotes for single quotes on keys
                        parsed = json.dumps(json.loads(jmodel), sort_keys=False, indent=2).replace('"type":','type:').replace('"database":','database:').replace('"schema":','schema:').replace('"name":','name:')

                        #creating .sqlx files
                        #write out to file in the appropriate location defined in variables
                        with open(f'{dataform_output_sources_path}/{schema}_{tables}.sqlx', "w") as jmodel_file:                            
                            print('Generated file: '+f'{dataform_output_sources_path}/{schema}_{tables}.sqlx')
                            jmodel_file.write("config "+parsed)
                            jmodel_file.close()


In [163]:
#Function used to convert DBT SQL model files to SQLX model files on dataform

def create_sqlx_models_files(model_files_path,dataform_output_models_path):
    #iterate on each SQL file on DBT directory and getting variables
    for filename in model_files_path:  
        single_file_name = os.path.basename(filename)
        destination_path = os.path.dirname(filename)
        destination_path = dataform_output_models_path+(destination_path.split('models')[1])+'/'
        destination_full_path = (destination_path+(os.path.basename(filename)).replace("sql","sqlx"))
        
     #######copying files to new directory##############   
        try:
            #copy files if directory already exists            
            shutil.copyfile(filename,destination_full_path)
        except IOError as e:
            # ENOENT(2): file does not exist, raised also on missing dest parent dir
            if e.errno != errno.ENOENT:
                raise
            # create directory if not exists           
            os.makedirs(destination_path)
            # copy SQL files to new directory          
            shutil.copyfile(filename,destination_full_path)
    #######end copying files to new directory##############

    
    #######reading file and replacing model syntax differences########
        # Read in the file
        with open(destination_full_path, 'r') as file :
            filedata = file.read()
            #getting header data to be replaced
            header=filedata[filedata.find("{{"):filedata.find("}}")+2]
            header_old=header
            #dictionary to replace simple syntax elements
            header_replace_dict = {"{{":"", "}}":"}", "materialized":"type","=":":","(":"{",")":"","'":'"',"unique_key":'uniqueKey'}

            #iterate through dictionary keys
            for key in header_replace_dict.keys():
                #replacing all headers based on dict mapping to 'header' variable
                header = header.replace(key, header_replace_dict[key])         


        #writing file with replaced header and models references
        with open(destination_full_path, 'w') as file:
            #writting on file the replaced header based on dictionary 
            new_file = filedata.replace(header_old,header)
            #converting transient tables to snowflake specific block
            new_file = re.sub(r'(transient)[ ]{0,9}[\:][ ]{0,9}(true|false|True|False|TRUE|FALSE)',r'snowflake: { \n     \1: \2 \n}',new_file)
            #converting disabled models syntax
            new_file = re.sub(r'(enabled)[ ]{0,9}[\:][ ]{0,9}(true|True|TRUE)','',new_file)
            new_file = re.sub(r'(enabled)[ ]{0,9}[\:][ ]{0,9}(false|False|FALSE)','disabled: true',new_file)
            #removing configurations that does not exists on dataform
            new_file = re.sub(r'(pre_hook|post_hook|alias|meta|persist_docs|merge_update_columns|on_schema_change)[ ]{0,9}[\:][ ]{0,9}.*[,}\")]','',new_file)
            #changing {{ref to ${ref
            new_file = re.sub(r"[{][{][ ]{0,6}(ref|REF|Ref|SOURCE|source|Source)[ ]{0,9}[(][ ]{0,9}[']?",'${ref("', new_file)     
            #including just 1 space before config header
            new_file = re.sub(r'(config)[ ]{0,40}[{]{0,40}','config {',new_file)
            #closing ) with ") after sources and refs
            new_file = re.sub(r"(['][)][ ]{0,9}[}])",'")',new_file)
            #removing alone commmas on start of config block
            new_file = re.sub(r"({)\n[ ]{0,100}[,]",'{',new_file)
            #changing syntax reference to macros from {{ to ${
            new_file = re.sub(r"(?<!\')({{)[ ]{0,20}[.]{0,20}",'${',new_file)
            new_file = re.sub(r"([\$].+)[ ]{0,9}(?<=\()(')",r'\1"',new_file)
            #changing ' to " on sources references
            new_file = re.sub(r"([\$].+)((?<=[0-9A-Za-z])[ ]{0,9}[_'][.]{0,10}[ ]{0,20}[,][.]{0,20}[ ]{0,20}[.]{0,20}['])",r'\1","',new_file)
            #changing invocation_id to snowflake uuid_string
            new_file = re.sub(r"('{{invocation_id}}'|{{invocation_id}})",'uuid_string()',new_file)
            #changing incremental loads macro
            new_file = re.sub(r"({[ ]{0,10}%[ ]{0,10}if[ ]{0,10}is_incremental.*)((\s*)(.*)\s*)({%[ ]{0,10}endif..*})",r'${ when(incremental(), \n`\4`) }',new_file)
            new_file = re.sub(r"((?:\${[ ]{0,10}when.*(\s.*)))(\${this\}\})",r'\1${self()}',new_file)
            
            

            try:
                file.write(new_file)                
                print('Generated file: '+destination_full_path)
            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))
    #######end reading file and replacing model syntax differences########
        


In [164]:
def create_sqlx_snapshot_files(snapshots_file_path,dataform_output_models_path):
    for filename in snapshots_file_path:  
        single_file_name = os.path.basename(filename)
        destination_path = os.path.dirname(filename)
        destination_path = dataform_output_models_path+'/snapshots'+(destination_path.split('snapshots')[1])+'/'
        destination_full_path = (destination_path+(os.path.basename(filename)).replace(".sql",".js"))

     #######copying files to new directory##############   
        try:
            #copy files if directory already exists

            shutil.copyfile(filename,destination_full_path)
        except IOError as e:
            # ENOENT(2): file does not exist, raised also on missing dest parent dir
            if e.errno != errno.ENOENT:
                raise
            # create directory if not exists
            os.makedirs(destination_path)
            # copy source files to new directory
            shutil.copyfile(filename,destination_full_path)
    #######end copying files to new directory##############

        #read copied files
        with open(destination_full_path, 'r') as file :
            filedata = file.read()   
            #getting the name of the snapshot source table
            table = re.search(r"({{)[ ]{0,9}(ref).*((?<=').*(?='))",filedata).group(3)
            #getting the name of the file and removing extension
            file_name = single_file_name.replace('.sql','')
            #removing macro references 
            filedata = re.sub(r'({%)[ ]{0,10}(endsnapshot|snapshot).*(})','',filedata)
            #if exists check cols (not supported to data form), replace for timestamp field
            filedata = re.sub(r"(check_cols.*[\]]{1,1}|check_cols.*[\"]{1,1})",'timestamp: '+dlh_timestamp_field,filedata)
            #getting header block
            filedata=filedata[filedata.find("{"):filedata.find("}")+2]
            #replacing fields on header block
            filedata = filedata.replace(')', 'source: {\n schema: "{{schema}}",\n name: "{{table}}",\n}, \n});')
            filedata = filedata.replace('{{schema}}',target_schema).replace('{{table}}',table)
            #create a dict with some syntax differences
            replace_dict = {"updated_at":"timestamp","unique_key":"  uniqueKey", "config":'scd("{{source_data_scd}}", ',"'":'"',"{{":"","}}":"","=":": "}
            
            #iterate through dictionary keys
            for key in replace_dict.keys():
                #replacing all headers
                filedata = filedata.replace(key, replace_dict[key]).replace('{{source_data_scd}}',file_name)
                filedata = re.sub(r'(scd\(.*)(\()',r'\1{',filedata)


        #writing file with replaced header and models references
        with open(destination_full_path, 'w') as file:
            #replacing files content
            new_file = filedata
            new_file = re.sub(r'(strategy)[ ]{0,9}[\:][ ]{0,9}.*(,)','',new_file)   
            #removing unsupported configurations
            new_file = re.sub(r'(target_database|target_schema|strategy|invalidate_hard_deletes|check_cols)[ ]{0,9}[\:][ ]{0,9}.*[,}\"\')]','',new_file)
            new_file = re.sub(r"[ ]{2,999}",'',new_file)
            new_file = re.sub(r"^\s*$",'',new_file,re.MULTILINE)
            new_file = re.sub(r'\n\s*\n','\n',new_file,re.MULTILINE)            
            #insert space to ident
            new_file = new_file.replace('uniqueKey:','  uniqueKey:').replace('timestamp:','  timestamp:').replace('source:','  source:').replace('schema:','   schema:').replace('name:','   name:').replace('},','  },')
            #create .JS snapshot file
            try:
                file.write('const scd = require("dataform-scd");\n\n')
                file.write(new_file)
                print('Generated file: '+destination_full_path)
            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))
                
    #######end reading file and replacing model syntax differences########
    

In [165]:
def dataform_install_configuration(dataform_root_path,dataform_credentials_file_path):
    
    try:
        shutil.rmtree(dataform_root_path)
    except OSError as err:
        print(err)

    #initiating dataform new project
    os.system("dataform init snowflake "+dataform_root_path)
    #copying snowflake credentials file
    shutil.copy(dataform_credentials_file_path,dataform_root_path)
    
    packages_file = dataform_root_path+ '/' +'package.json'
        
    with open(packages_file, 'r') as file:
        packages_file_data = file.read()

In [166]:
def dataform_assertions_documentation(schema_files_path,dataform_output_models_path):

    for file in schema_files_path:
        #reading all YAML files on source directory
        dic = read_yaml_file(file)
        #getting only sources entity from yaml file
        if 'models' in dic.keys():

            models_dir = dic["models"]
            dictionary={}
            #print(models_dir)
            unique_list=[]
            not_null_list=[]
            not_null = {} 
            table_descriptions = {} 
            description={}
            unique = {}  
            tables_list=[]
            description_list=[]
            table_description_list=[]
            jmodel_tables_description=''
            jmodel_description=''
            jmodel=''
            for model in models_dir:
                #declaring variables for each element of dictionary that will be used. tables = list, schema and database = str        
                #print(model["name"])
                if 'columns' in model:
                    columns = (model["columns"]) 
                if 'name' in model:
                    tables = (model["name"])


                tables_list.append(tables)
                #getting tables descriptions
                if 'description' in model: 
                    descriptions = (model["description"])
                    table_descriptions = {descriptions:tables}
                    table_description_list.append(table_descriptions.copy())


                #getting column tests and descriptions
                for column in columns:
                    tests= (column.get('tests'))
                    column_get = column.get('name')           
                    descriptions= (column.get('description'))

                    #print(tests)

                    if 'unique' in tests:
                        unique = {column_get:tables}                                                        
                        unique_list.append(unique.copy())


                    if 'not_null' in tests:
                        not_null = {column_get:tables}
                        not_null_list.append(not_null.copy())


                    if 'description'in column:
                        description = {'|    '+column_get+': '+"'"+descriptions+"'":tables}
                        description_list.append(description.copy())


            #print(not_null_list)
            
            keyfunc = lambda d: next(iter(d.values()))   

            print('Found table description' +str(table_description_list)+'\n')
            print('Found uniqueKey test' +str(unique_list)+'\n')
            print('Found Not_Null test' +str(not_null_list)+'\n')
            print('Found column description' +str(description_list)+'\n')
            
            #create dictionary for data tests and tables/columns descriptions
            not_null_dict={k: [x for d in g for x in d] 
                for k, g in itertools.groupby(sorted(not_null_list, key=keyfunc), key=keyfunc)}

            unique_dict={k: [x for d in g for x in d] 
                for k, g in itertools.groupby(sorted(unique_list, key=keyfunc), key=keyfunc)}

            description_dict={k: [x for d in g for x in d] 
                for k, g in itertools.groupby(sorted(description_list, key=keyfunc), key=keyfunc)}

            table_description_dict={k: [x for d in g for x in d] 
                    for k, g in itertools.groupby(sorted(table_description_list, key=keyfunc), key=keyfunc)}


            # list of all .sql files in a directory
            dataform_files = find_files(dataform_output_models_path, '*.sqlx')


            files_list =[]
            file_name_list =[]
            
            #getting all model files on dataform path
            for file in dataform_files:
                #only tables that has descriptions or tests
                if os.path.basename(file).replace('.sqlx','') in tables_list:
                    file_name = os.path.basename(file)
                    destination_path = os.path.dirname(file)
                    full_path = destination_path+'/'+file_name     
                    files_list.append(file_name.replace('.sqlx',''))


                    #read files
                    with open(full_path, 'r') as file :

                        file_name = file_name.replace('.sqlx','')
                        #add description to table (setting variable)
                        if file_name in table_description_dict.keys():
                            jmodel_tables_description='  description: '+str(table_description_dict[file_name]).replace("'",'"').replace('[','').replace(']','')+',\n'

                        #add assertions { on variable
                        filedata = file.read()
                        if file_name in unique_dict.keys() or file_name in unique_dict.keys():
                            jmodel='  assertions: {\n'


                            #when exists uniqueKey test on dbt, create on dataform
                            if file_name in unique_dict.keys():                    
                                jmodel+='    uniqueKey: '+str(unique_dict[file_name]).replace("'",'"')+',\n'

                            #when exists NotNull test on dbt, create on dataform
                            if file_name in not_null_dict.keys():
                                jmodel+='    nonNull: '+str(not_null_dict[file_name]).replace("'",'"')+'\n'


                            jmodel+='\n  }\n}'

                        #add column description on dataform
                        if file_name in description_dict.keys():         
                            jmodel_description = '  columns: {\n'
                            jmodel_description += str(description_dict[file_name]).replace('"','').replace("'",'"').replace('[','').replace(']','').replace('|','\n')
                            jmodel_description += '\n  }\n'


                        #setting variable with all tests and description to be added on model
                        header=filedata[filedata.find("{"):filedata.find("}")+2]                
                        header_old=header
                        header = header.replace('}','')+jmodel_tables_description+jmodel_description+jmodel

        
                        with open(f'{full_path}', "w") as jmodel_file:
                                #replace old header to new header with assertions and column or table documentations
                                jmodel_file.write(filedata.replace(header_old,header))


In [167]:
# list of all .sql files in a directory
model_files = find_files(dbt_models_file_path, '*.sql')
# list of all .sql files in a directory
snapshot_files = find_files(dbt_snapshots_file_path, '*.sql')
# list of test and documentation files
source_files = find_files(dbt_models_file_path, '*.yml')
# list of test and documentation files
schema_files = find_files(dbt_models_file_path, '*.yml')
# dataform output files
dataform_output_models_path = "../_dataform_output/definitions"

def dbt_dataform_converter(dataform_root_path,model_files,snapshot_files,source_files,schema_files,dataform_output_models_path):
    
    
    #installing dataform
    dataform_install_configuration(dataform_root_path,dataform_credentials_file_path)    
    #creating source
    create_sqlx_json_source_file(source_files,dataform_output_sources_path)
    #creating models
    create_sqlx_models_files(model_files,dataform_output_models_path)
    #creating snapshot files
    create_sqlx_snapshot_files(snapshot_files,dataform_output_models_path)
    #addind test to models
    dataform_assertions_documentation(schema_files,dataform_output_models_path)

    
    

In [168]:
dbt_dataform_converter(dataform_root_path,model_files,snapshot_files,source_files,schema_files,dataform_output_models_path)

Writing project files...

Directories successfully created:
  /Users/guilhermealcantara/OneDrive/brf consulting/dbt - dataform converter/_dataform_output
  /Users/guilhermealcantara/OneDrive/brf consulting/dbt - dataform converter/_dataform_output/definitions
  /Users/guilhermealcantara/OneDrive/brf consulting/dbt - dataform converter/_dataform_output/includes
Files successfully written:
  /Users/guilhermealcantara/OneDrive/brf consulting/dbt - dataform converter/_dataform_output/dataform.json
  /Users/guilhermealcantara/OneDrive/brf consulting/dbt - dataform converter/_dataform_output/package.json
  /Users/guilhermealcantara/OneDrive/brf consulting/dbt - dataform converter/_dataform_output/.gitignore
NPM packages successfully installed.
Generated file: ../_dataform_output/definitions/sources/DEMO_SQUARE_ALT13_CUSTOMER.sqlx
Generated file: ../_dataform_output/definitions/sources/DEMO_SQUARE_ALT13_ORDER_DISCOUNT.sqlx
Generated file: ../_dataform_output/definitions/sources/DEMO_SQUARE_AL